# Report of Project 1: Navigation

## Learning algorithm

TODO: The report clearly describes the learning algorithm, along with the chosen hyperparameters. It also describes the model architectures for any neural networks.

## Plot of rewards

TODO: A plot of rewards per episode is included to illustrate that the agent is able to receive an average reward (over 100 episodes) of at least +13. The submission reports the number of episodes needed to solve the environment.

![Scores](scores.png "Scores")

## Ideas for future work

TODO: The submission has concrete future ideas for improving the agent's performance.